# EMOTIW19 - FUSION MODELS

## 1. Setup Project

In [1]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import pandas as pd, tqdm
import matplotlib.pyplot as plt
import numpy as np, cv2, glob, argparse
from matplotlib.patches import Rectangle
from IPython import display

import os, sys, re
root_dir = '/mnt/ubuntu1/dntai/projects_merge/EmotiWPrj/'
lib_dirs = [os.path.join(root_dir, 'sources')]
for lib in lib_dirs:
    if lib not in sys.path: sys.path.insert(0, lib)
# np.set_printoptions(precision=2, suppress=True, formatter={'float': '{: 0.4f}'.format}, linewidth=1000)

In [2]:
from emotiw19prj.fusion.common import *

Using TensorFlow backend.


In [11]:
# Parameters
parser = argparse.ArgumentParser(description='Facial Emotion Recognition - EmotiW2019')
parser.add_argument('--gpus',   type=str, default="['3']", help='')

parser.add_argument('--batch_size', type=int, default=8, help='')

parser.add_argument('--frame_root_path' , type=str  , default="/mnt/ubuntu1/dntai/projects/datasets/emotiw/AFEW19/frames", help='')
parser.add_argument('--db_file' , type=str  , default="/mnt/ubuntu1/dntai/projects/datasets/emotiw/AFEW19/emotiw19_trainvalid_person_png.h5", help='')

parser.add_argument('--result_dir', default=root_dir + "data/results/emotiw19_fusion", 
                    type=str, metavar='result_dir', help='')

# # Audio Features 
parser.add_argument('--audio_features_dir', default=root_dir + "data/results/feature3d/densenet_scale124_mean_190621_01", 
                    type=str, metavar='runtime_dir', help='')

args, _ = parser.parse_known_args()
params  = vars(args)
params["gpus"] = eval(params["gpus"])
print("Parameters: ")
for key in params.keys(): print("+ %s: %s"%(key,params[key]))

Parameters: 
+ frame_root_path: /mnt/ubuntu1/dntai/projects/datasets/emotiw/AFEW19/frames
+ db_file: /mnt/ubuntu1/dntai/projects/datasets/emotiw/AFEW19/emotiw19_trainvalid_person_png.h5
+ batch_size: 8
+ result_dir: /mnt/ubuntu1/dntai/projects_merge/EmotiWPrj/data/results/emotiw19_fusion
+ gpus: ['3']


In [4]:
# Choose gpus, init session
choose_keras_environment(gpus = params["gpus"], keras_backend = "tensorflow", verbose = 1)
init_session()
check_tensorflow_environment()

Environment GPUs:
+ Choose GPUs:  3
+ Keras backend:  tensorflow
TensorFlow Version: 1.9.0
Default GPU Device: /device:GPU:0


## 2. Load Data

In [18]:
emotiw_name = np.array(["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"])
emotiw_sdict = dict(zip(emotiw_name, np.arange(len(emotiw_name))))

df_all = pd.HDFStore(params["db_file"], "r")
print(df_all.keys())
df_emotion = df_all["emotion"]
df_video   = df_all["video"]
df_frames  = df_all["frames"]
df_faces   = df_all["faces"]
df_person  = df_all["person"]
df_all.close()

df_train = df_video.query("type=='train'")
df_valid = df_video.query("type=='valid'")

print("Number of training: ", len(df_train))
print("Number of validating: ", len(df_valid))

['/emotion', '/faces', '/frames', '/person', '/video']
Number of training:  773
Number of validating:  383


In [41]:
print("Validating: ", dict(df_valid.groupby(["emotion"])["name"].count().reset_index().values))
print("Training: ", dict(df_train.groupby(["emotion"])["name"].count().reset_index().values))

Validating:  {'Happy': 63, 'Sad': 61, 'Neutral': 63, 'Surprise': 46, 'Angry': 64, 'Disgust': 40, 'Fear': 46}
Training:  {'Happy': 150, 'Sad': 117, 'Neutral': 144, 'Surprise': 74, 'Angry': 133, 'Disgust': 74, 'Fear': 81}


In [49]:
x_train_data = df_train["name"].values
y_train_data = np.array([emotiw_sdict[v] for v in df_train["emotion"].values])
x_valid_data = df_valid["name"].values
y_valid_data = np.array([emotiw_sdict[v] for v in df_valid["emotion"].values])

idx_train_video_sorted = np.argsort(x_train_data)
train_video_names      = x_train_data[idx_train_video_sorted]
train_emotiw_data      = y_train_data[idx_train_video_sorted]

idx_valid_video_sorted = np.argsort(x_valid_data)
valid_video_names      = x_valid_data[idx_valid_video_sorted]
valid_emotiw_data      = y_valid_data[idx_valid_video_sorted]

train_name_s2i = dict(zip(train_video_names, range(len(train_video_names))))
valid_name_s2i = dict(zip(valid_video_names, range(len(valid_video_names))))

print("Train: ", train_video_names[0:4], " ...")
print("Valid: ", valid_video_names[0:4], " ...")

Train:  ['000029301' '000046280' '000059880' '000102534']  ...
Valid:  ['000123880' '000142325' '000147200' '000149120']  ...


## 3. Audio Features